In [92]:
import sys

%load_ext autoreload
%autoreload 2

sys.path.append("..")

from src.make_predictions import *
from src.data.process_data import *

import sqlite3
import joblib
import pandas as pd

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [93]:
final_stacked_model = joblib.load('../models/finalized_stacked_model')

prev_season_elo = pd.read_csv("../data/clean/final_elo_ratings_2020.csv")

In [94]:
connection = sqlite3.connect('../data/nba.db')

df = load_current_season_team_data(connection, 2021)



In [95]:
pd.options.display.max_columns=100
df.loc[54012:54013]

,SEASON,TEAM_ID,TEAM_ABBREVIATION,TEAM_NAME,GAME_ID,GAME_DATE,MATCHUP,WL,MIN,FGM,FGA,FG_PCT,FG3M,FG3A,FG3_PCT,FTM,FTA,FT_PCT,OREB,DREB,REB,AST,STL,BLK,TOV,PF,PTS,PLUS_MINUS,E_OFF_RATING,OFF_RATING,E_DEF_RATING,DEF_RATING,E_NET_RATING,NET_RATING,AST_PCT,AST_TOV,AST_RATIO,OREB_PCT,DREB_PCT,REB_PCT,E_TM_TOV_PCT,TM_TOV_PCT,EFG_PCT,TS_PCT,USG_PCT,E_USG_PCT,E_PACE,PACE,PACE_PER40,POSS,PIE,PCT_FGA_2PT,PCT_FGA_3PT,PCT_PTS_2PT,PCT_PTS_2PT_MR,PCT_PTS_3PT,PCT_PTS_FB,PCT_PTS_FT,PCT_PTS_OFF_TOV,PCT_PTS_PAINT,PCT_AST_2PM,PCT_UAST_2PM,PCT_AST_3PM,PCT_UAST_3PM,PCT_AST_FGM,PCT_UAST_FGM
54012,2021-22,1610612750,MIN,Minnesota Timberwolves,0022100169,2021-11-10,MIN @ GSW,L,240,39,95,0.411,17,44,0.386,15,20,0.75,17,23,40,30,8,4,17,28,110,-13,106.0,110.0,124.7,121.8,-18.8,-11.8,0.769,1.76,19.8,0.400,0.511,0.449,16.378,17.0,0.495,0.525,1.0,0.200,101.2,100.5,83.75,100.0,0.393,0.542,0.458,0.400,0.018,0.464,0.127,0.136,0.164,0.382,0.636,0.364,0.941,0.059,0.769,0.231
54013,2021-22,1610612744,GSW,Golden State Warriors,0022100169,2021-11-10,GSW vs. MIN,W,240,40,85,0.471,11,40,0.275,32,40,0.80,20,33,53,23,14,6,16,21,123,13,124.7,121.8,106.0,110.0,18.8,11.8,0.575,1.44,16.2,0.489,0.600,0.551,16.227,15.8,0.535,0.599,1.0,0.204,101.2,100.5,83.75,101.0,0.607,0.529,0.471,0.472,0.081,0.268,0.195,0.260,0.146,0.390,0.483,0.517,0.818,0.182,0.575,0.425


In [96]:
home_team = 'ATL'
away_team = 'BOS'
home_spread = -4
away_spread = -home_spread
home_ml = 1.57
away_ml = 2.55

conn = sqlite3.connect('../data/nba.db')
season = 2021

df = load_current_season_team_data(conn, season)

df = clean_team_data(df)

df = prep_for_aggregation(df)

spreads, moneylines = load_current_season_betting_data(conn, 2021)
clean_moneylines = clean_moneyline_df(df = moneylines)
clean_spreads = clean_spreads_df(df = spreads)

full_df = merge_betting_and_boxscore_data(clean_spreads, clean_moneylines, df)
full_df = create_matchups(full_df)
full_df = get_team_and_opp_ewm(full_df, min_periods=1, alpha=0.1)

full_df = add_percentage_features(full_df)
full_df = add_rest_days_for_model(full_df)

current_elo, full_df = get_current_season_elo_ratings(full_df, prev_season_elo)

matchup = create_matchup_rows_for_model(full_df, home_team, away_team,
                                                home_spread, away_spread, home_ml, away_ml)

matchup

..\src\data\process_data.py:145: RuntimeWarning: All-NaN axis encountered
  abs(row)) if np.nanmax(row) > 0 else -np.nanmax(abs(row)), axis=1)
..\src\data\process_data.py:151: RuntimeWarning: All-NaN axis encountered
  abs(row)) if np.nanmax(row) > 0 else -np.nanmax(abs(row)), axis=1)
..\src\data\process_data.py:212: RuntimeWarning: All-NaN axis encountered
  lambda row: -np.nanmax(abs(row)) if np.nanmax(row) < 0 else np.nanmax(abs(row)), axis=1)
..\src\data\process_data.py:217: RuntimeWarning: All-NaN axis encountered
  lambda row: -np.nanmax(abs(row)) if np.nanmax(row) < 0 else np.nanmax(abs(row)), axis=1)
Progress: 100%|██████████████████████████████████████████████████████████████████████████| 1/1 [00:01<00:00,  1.02s/it]
progress:: 688it [00:00, 3230.15it/s]


,RECORD_team_diff,FG2M_team_diff,FG2A_team_diff,FG3M_team_diff,FG3A_team_diff,FTM_team_diff,FTA_team_diff,OREB_team_diff,DREB_team_diff,REB_team_diff,AST_team_diff,STL_team_diff,BLK_team_diff,TOV_team_diff,PF_team_diff,PTS_team_diff,PLUS_MINUS_team_diff,E_OFF_RATING_team_diff,E_DEF_RATING_team_diff,E_NET_RATING_team_diff,POSS_team_diff,PIE_team_diff,PTS_2PT_MR_team_diff,PTS_FB_team_diff,PTS_OFF_TOV_team_diff,PTS_PAINT_team_diff,AST_2PM_team_diff,AST_3PM_team_diff,UAST_2PM_team_diff,UAST_3PM_team_diff,ATS_DIFF_team_diff,RECORD_opp_diff,FG2M_opp_diff,FG2A_opp_diff,FG3M_opp_diff,FG3A_opp_diff,FTM_opp_diff,FTA_opp_diff,OREB_opp_diff,DREB_opp_diff,REB_opp_diff,AST_opp_diff,STL_opp_diff,BLK_opp_diff,TOV_opp_diff,PF_opp_diff,PTS_opp_diff,PLUS_MINUS_opp_diff,E_OFF_RATING_opp_diff,E_DEF_RATING_opp_diff,E_NET_RATING_opp_diff,POSS_opp_diff,PIE_opp_diff,PTS_2PT_MR_opp_diff,PTS_FB_opp_diff,PTS_OFF_TOV_opp_diff,PTS_PAINT_opp_diff,AST_2PM_opp_diff,AST_3PM_opp_diff,UAST_2PM_opp_diff,UAST_3PM_opp_diff,ATS_DIFF_opp_diff,TEAM_COVERED_opp_diff,WL_PCT_diff,ATS_PCT_diff,AVG_SPREAD_diff,AVG_ML_diff,AVG_ATS_DIFF_diff,REST_diff,OREB_PCT_team_diff,OREB_PCT_opp_diff,DREB_PCT_team_diff,DREB_PCT_opp_diff,REB_PCT_team_diff,REB_PCT_opp_diff,TS_PCT_team_diff,TS_PCT_opp_diff,EFG_PCT_team_diff,EFG_PCT_opp_diff,AST_RATIO_team_diff,AST_RATIO_opp_diff,TOV_PCT_team_diff,TOV_PCT_opp_diff,team_elo_pred,team_rating_i,opp_rating_i,elo_MOV_pred,SPREAD_team,ML_team
0,0.09927,3.014749,3.994226,0.731729,-3.191034,-2.587251,-1.866142,-0.163139,-0.524043,-0.687183,2.102338,0.017554,-0.053954,-1.257756,-1.903728,5.637434,4.044544,7.009676,2.051243,4.957773,-1.209466,0.007633,2.02489,0.909453,0.239528,3.869878,2.760408,-0.854229,0.254737,1.352355,3.052637,-0.09927,3.851445,4.276921,-1.089699,-0.44355,-2.840902,-3.553864,1.257865,-3.328125,-2.070261,3.336768,0.692587,-0.37008,-1.349763,-1.404348,1.59289,-4.044544,2.051243,7.009676,-4.957773,-0.061054,-0.007633,-0.995981,0.169172,-0.738795,8.169716,2.287604,1.132447,1.50322,-2.131612,-3.052637,-0.15307,0.023715,0.019763,-0.983766,-0.050834,0.70671,1.0,0.013778,0.023878,-0.023878,-0.013778,0.007933,-0.007933,0.027995,-0.010214,0.041957,0.002741,2.435038,3.423681,-1.023045,-1.331241,0.689109,1583.720684,1514.448540,4.94,-4.0,1.57
1,-0.09927,-3.014749,-3.994226,-0.731729,3.191034,2.587251,1.866142,0.163139,0.524043,0.687183,-2.102338,-0.017554,0.053954,1.257756,1.903728,-5.637434,-4.044544,-7.009676,-2.051243,-4.957773,1.209466,-0.007633,-2.02489,-0.909453,-0.239528,-3.869878,-2.760408,0.854229,-0.254737,-1.352355,-3.052637,0.09927,-3.851445,-4.276921,1.089699,0.44355,2.840902,3.553864,-1.257865,3.328125,2.070261,-3.336768,-0.692587,0.37008,1.349763,1.404348,-1.59289,4.044544,-2.051243,-7.009676,4.957773,0.061054,0.007633,0.995981,-0.169172,0.738795,-8.169716,-2.287604,-1.132447,-1.50322,2.131612,3.052637,0.15307,-0.023715,-0.019763,0.983766,0.050834,-0.70671,-1.0,-0.013778,-0.023878,0.023878,0.013778,-0.007933,0.007933,-0.027995,0.010214,-0.041957,-0.002741,-2.435038,-3.423681,1.023045,1.331241,0.310891,1514.448540,1583.720684,-4.94,4.0,2.55


In [97]:
connection = sqlite3.connect('../data/nba.db')


main(connection, 2021, final_stacked_model, prev_season_elo, append=True)

progress: 100%|██████████████████████████████████████████████████████████████████████████| 1/1 [00:09<00:00,  9.47s/it]
..\src\data\process_data.py:145: RuntimeWarning: All-NaN axis encountered
  abs(row)) if np.nanmax(row) > 0 else -np.nanmax(abs(row)), axis=1)
..\src\data\process_data.py:151: RuntimeWarning: All-NaN axis encountered
  abs(row)) if np.nanmax(row) > 0 else -np.nanmax(abs(row)), axis=1)
..\src\data\process_data.py:212: RuntimeWarning: All-NaN axis encountered
  lambda row: -np.nanmax(abs(row)) if np.nanmax(row) < 0 else np.nanmax(abs(row)), axis=1)
..\src\data\process_data.py:217: RuntimeWarning: All-NaN axis encountered
  lambda row: -np.nanmax(abs(row)) if np.nanmax(row) < 0 else np.nanmax(abs(row)), axis=1)
Progress: 100%|██████████████████████████████████████████████████████████████████████████| 1/1 [00:01<00:00,  1.95s/it]
progress:: 696it [00:00, 4579.11it/s]


,game_date,home_team,away_team,home_spread,home_cover_probs,away_cover_probs,home_cover_avg
0,2021-12-06,DET,OKC,-3.0,0.496469,0.503966,0.496252
1,2021-12-06,IND,WAS,-5.5,0.509115,0.490469,0.509323
2,2021-12-06,CHA,PHI,7.5,0.510683,0.490421,0.510131
3,2021-12-06,MIA,MEM,-6.0,0.473984,0.524575,0.474704
4,2021-12-06,CHI,DEN,-2.0,0.509730,0.488905,0.510413
5,2021-12-06,MIL,CLE,-6.5,0.506613,0.493539,0.506537
6,2021-12-06,MIN,ATL,-2.0,0.506005,0.493153,0.506426
7,2021-12-06,PHX,SAS,-7.0,0.479914,0.520752,0.479581
8,2021-12-06,GSW,ORL,-15.5,0.507276,0.496199,0.505538
9,2021-12-06,POR,LAC,3.0,0.522463,0.476143,0.523160


In [98]:
final_stacked_model_sfs = joblib.load('../models/finalized_stacked_model_sfs')

connection = sqlite3.connect('../data/nba.db')


main(connection, 2021, final_stacked_model_sfs, prev_season_elo, append=False)


updating basic team boxscores
updating advanced team boxscores
num_games_updated: 0
All team advanced boxscores up to date in season 2021-22
updating scoring boxscores
num_games_updated: 0
All team advanced boxscores up to date in season 2021-22
updating moneyline data
moneyline table is up to date
updating spreads data
Updating spreads for 0 days
spreads table is up to date


..\src\data\process_data.py:145: RuntimeWarning: All-NaN axis encountered
  abs(row)) if np.nanmax(row) > 0 else -np.nanmax(abs(row)), axis=1)
..\src\data\process_data.py:151: RuntimeWarning: All-NaN axis encountered
  abs(row)) if np.nanmax(row) > 0 else -np.nanmax(abs(row)), axis=1)
..\src\data\process_data.py:212: RuntimeWarning: All-NaN axis encountered
  lambda row: -np.nanmax(abs(row)) if np.nanmax(row) < 0 else np.nanmax(abs(row)), axis=1)
..\src\data\process_data.py:217: RuntimeWarning: All-NaN axis encountered
  lambda row: -np.nanmax(abs(row)) if np.nanmax(row) < 0 else np.nanmax(abs(row)), axis=1)
Progress: 100%|██████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00,  1.11it/s]
progress:: 696it [00:00, 4936.31it/s]


,game_date,home_team,away_team,home_spread,home_cover_probs,away_cover_probs,home_cover_avg
0,2021-12-06,DET,OKC,-3.0,0.500124,0.515288,0.492418
1,2021-12-06,IND,WAS,-5.5,0.508243,0.504018,0.502113
2,2021-12-06,CHA,PHI,7.5,0.518305,0.497921,0.510192
3,2021-12-06,MIA,MEM,-6.0,0.502746,0.509546,0.496600
4,2021-12-06,CHI,DEN,-2.0,0.521493,0.495891,0.512801
5,2021-12-06,MIL,CLE,-6.5,0.518165,0.491329,0.513418
6,2021-12-06,MIN,ATL,-2.0,0.535593,0.481934,0.526830
7,2021-12-06,PHX,SAS,-7.0,0.476062,0.535938,0.470062
8,2021-12-06,GSW,ORL,-15.5,0.505112,0.513429,0.495842
9,2021-12-06,POR,LAC,3.0,0.544748,0.475544,0.534602


In [99]:
final_stacked_model_sfs = joblib.load('../models/finalized_stacked_model_retrain1')

connection = sqlite3.connect('../data/nba.db')


main(connection, 2021, final_stacked_model_sfs, prev_season_elo, append=False)


updating basic team boxscores
updating advanced team boxscores
num_games_updated: 0
All team advanced boxscores up to date in season 2021-22
updating scoring boxscores
num_games_updated: 0
All team advanced boxscores up to date in season 2021-22
updating moneyline data
moneyline table is up to date
updating spreads data
Updating spreads for 0 days
spreads table is up to date


..\src\data\process_data.py:145: RuntimeWarning: All-NaN axis encountered
  abs(row)) if np.nanmax(row) > 0 else -np.nanmax(abs(row)), axis=1)
..\src\data\process_data.py:151: RuntimeWarning: All-NaN axis encountered
  abs(row)) if np.nanmax(row) > 0 else -np.nanmax(abs(row)), axis=1)
..\src\data\process_data.py:212: RuntimeWarning: All-NaN axis encountered
  lambda row: -np.nanmax(abs(row)) if np.nanmax(row) < 0 else np.nanmax(abs(row)), axis=1)
..\src\data\process_data.py:217: RuntimeWarning: All-NaN axis encountered
  lambda row: -np.nanmax(abs(row)) if np.nanmax(row) < 0 else np.nanmax(abs(row)), axis=1)
Progress: 100%|██████████████████████████████████████████████████████████████████████████| 1/1 [00:01<00:00,  1.50s/it]
progress:: 696it [00:00, 4218.32it/s]


,game_date,home_team,away_team,home_spread,home_cover_probs,away_cover_probs,home_cover_avg
0,2021-12-06,DET,OKC,-3.0,0.496375,0.503290,0.496543
1,2021-12-06,IND,WAS,-5.5,0.501361,0.495903,0.502729
2,2021-12-06,CHA,PHI,7.5,0.509034,0.489200,0.509917
3,2021-12-06,MIA,MEM,-6.0,0.501051,0.494517,0.503267
4,2021-12-06,CHI,DEN,-2.0,0.506041,0.492077,0.506982
5,2021-12-06,MIL,CLE,-6.5,0.507409,0.489488,0.508961
6,2021-12-06,MIN,ATL,-2.0,0.525106,0.474286,0.525410
7,2021-12-06,PHX,SAS,-7.0,0.467453,0.527751,0.469851
8,2021-12-06,GSW,ORL,-15.5,0.487827,0.515794,0.486017
9,2021-12-06,POR,LAC,3.0,0.531588,0.471859,0.529864


In [100]:
final_stacked_model_sfs = joblib.load('../models/finalized_stacked_model_retrain-21-11-26')

connection = sqlite3.connect('../data/nba.db')


main(connection, 2021, final_stacked_model_sfs, prev_season_elo, append=False)


updating basic team boxscores
updating advanced team boxscores
num_games_updated: 0
All team advanced boxscores up to date in season 2021-22
updating scoring boxscores
num_games_updated: 0
All team advanced boxscores up to date in season 2021-22
updating moneyline data
moneyline table is up to date
updating spreads data
Updating spreads for 0 days
spreads table is up to date


..\src\data\process_data.py:145: RuntimeWarning: All-NaN axis encountered
  abs(row)) if np.nanmax(row) > 0 else -np.nanmax(abs(row)), axis=1)
..\src\data\process_data.py:151: RuntimeWarning: All-NaN axis encountered
  abs(row)) if np.nanmax(row) > 0 else -np.nanmax(abs(row)), axis=1)
..\src\data\process_data.py:212: RuntimeWarning: All-NaN axis encountered
  lambda row: -np.nanmax(abs(row)) if np.nanmax(row) < 0 else np.nanmax(abs(row)), axis=1)
..\src\data\process_data.py:217: RuntimeWarning: All-NaN axis encountered
  lambda row: -np.nanmax(abs(row)) if np.nanmax(row) < 0 else np.nanmax(abs(row)), axis=1)
Progress: 100%|██████████████████████████████████████████████████████████████████████████| 1/1 [00:01<00:00,  1.25s/it]
progress:: 696it [00:00, 4640.59it/s]


,game_date,home_team,away_team,home_spread,home_cover_probs,away_cover_probs,home_cover_avg
0,2021-12-06,DET,OKC,-3.0,0.497240,0.506517,0.495361
1,2021-12-06,IND,WAS,-5.5,0.503245,0.497137,0.503054
2,2021-12-06,CHA,PHI,7.5,0.513429,0.490190,0.511619
3,2021-12-06,MIA,MEM,-6.0,0.504156,0.497816,0.503170
4,2021-12-06,CHI,DEN,-2.0,0.508987,0.491563,0.508712
5,2021-12-06,MIL,CLE,-6.5,0.508839,0.489619,0.509610
6,2021-12-06,MIN,ATL,-2.0,0.527596,0.471656,0.527970
7,2021-12-06,PHX,SAS,-7.0,0.471433,0.527512,0.471960
8,2021-12-06,GSW,ORL,-15.5,0.500831,0.507950,0.496440
9,2021-12-06,POR,LAC,3.0,0.534388,0.469957,0.532216


In [101]:
final_stacked_model_sfs = joblib.load('../models/finalized_stacked_model_retrain-21-12-3')

connection = sqlite3.connect('../data/nba.db')


main(connection, 2021, final_stacked_model_sfs, prev_season_elo, append=False)


updating basic team boxscores
updating advanced team boxscores
num_games_updated: 0
All team advanced boxscores up to date in season 2021-22
updating scoring boxscores
num_games_updated: 0
All team advanced boxscores up to date in season 2021-22
updating moneyline data
moneyline table is up to date
updating spreads data
Updating spreads for 0 days
spreads table is up to date


..\src\data\process_data.py:145: RuntimeWarning: All-NaN axis encountered
  abs(row)) if np.nanmax(row) > 0 else -np.nanmax(abs(row)), axis=1)
..\src\data\process_data.py:151: RuntimeWarning: All-NaN axis encountered
  abs(row)) if np.nanmax(row) > 0 else -np.nanmax(abs(row)), axis=1)
..\src\data\process_data.py:212: RuntimeWarning: All-NaN axis encountered
  lambda row: -np.nanmax(abs(row)) if np.nanmax(row) < 0 else np.nanmax(abs(row)), axis=1)
..\src\data\process_data.py:217: RuntimeWarning: All-NaN axis encountered
  lambda row: -np.nanmax(abs(row)) if np.nanmax(row) < 0 else np.nanmax(abs(row)), axis=1)
Progress: 100%|██████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00,  1.03it/s]
progress:: 696it [00:00, 5354.07it/s]


,game_date,home_team,away_team,home_spread,home_cover_probs,away_cover_probs,home_cover_avg
0,2021-12-06,DET,OKC,-3.0,0.471169,0.481148,0.495011
1,2021-12-06,IND,WAS,-5.5,0.478839,0.475409,0.501715
2,2021-12-06,CHA,PHI,7.5,0.490925,0.465455,0.512735
3,2021-12-06,MIA,MEM,-6.0,0.475975,0.475529,0.500223
4,2021-12-06,CHI,DEN,-2.0,0.481304,0.469136,0.506084
5,2021-12-06,MIL,CLE,-6.5,0.479833,0.472362,0.503736
6,2021-12-06,MIN,ATL,-2.0,0.499738,0.453058,0.523340
7,2021-12-06,PHX,SAS,-7.0,0.442823,0.510421,0.466201
8,2021-12-06,GSW,ORL,-15.5,0.473557,0.490666,0.491446
9,2021-12-06,POR,LAC,3.0,0.511279,0.447290,0.531995
